In [5]:
#imports
import pandas as pd
import numpy as np
from pylab import *
import string
from unidecode import unidecode

#load csv
veg_data_to_load = "zomato.csv"
veg_data = pd.read_csv(veg_data_to_load, encoding='ISO-8859-1')
veg_data_df = pd.DataFrame(veg_data).reset_index()



In [87]:
# Creating the dataframe from desired columns
veg_data_df = veg_data_df[["Restaurant Name","Address", "City", "Aggregate rating", "Votes"]]


In [7]:
#Cleaning the data.  Removing non-alphanumeric columns.  Don't include spaces
veg_data_df["Restaurant Name"] = veg_data_df["Restaurant Name"].str.replace('[^a-zA-Z ]', '')
#Cleaning the data.  Removing non-alphanumeric columns.  Don't include commas or spaces
veg_data_df["Address"] = veg_data_df["Address"].str.replace('[^a-zA-Z],', '')
#Cleaning the data.  Removing non-alphanumeric columns.  Don't include spaces
veg_data_df["City"] = veg_data_df["City"].str.replace('[^a-zA-Z] ', '')
veg_data_df = veg_data_df.rename(columns = {"Aggregate rating":"Rating"})
veg_data_df = veg_data_df.rename(columns = {"Votes":"Reviewers Count"})

In [74]:
#BRING IN YELP

#import
import requests
import pandas as pd
 
# Developer API key
API_KEY= "mk-Wsg9uQeXyKH3sa02giTl_xVqw3pC3cDNZna5OYlYPz27WIcX24kOxWXYD6d3XnQvqlOvGrZvCwTpC5Wl1jUZklx6ZX0q3FOqOs_KZ5wjLpHZC3u1XQk2ZLRRmXHYx" 
 
# What you are searching for
DEFAULT_TERM = 'Vegetarian'
# Business location

#Yelp has a bug that will only return 50 records at a time.
# to work around this, I am running once with the original list definitions,
# then appending.  I am using the following cities/locations:
# Los Angeles, San Francisco, New York, Seattle, Austin, Chicago, Portland, Pittsburgh, Albany, St. Louis, Minneapolis,
# Washington, Houston, Philadelphia, San Diego, Phoenix, San Jose, Boston, Baltimore, Fresno, Florida, New Mexico 

DEFAULT_LOCATION = 'New Mexico'
# Maximum number of results to return
SEARCH_LIMIT = 50
OFFSETN = 51
 
# Busines search end point
url = 'https://api.yelp.com/v3/businesses/search'
# Header should contain the API key
headers = {'Authorization': 'Bearer {}'.format(API_KEY)}
# Search parameters
url_params = {
	'term': DEFAULT_TERM, 
	'location': DEFAULT_LOCATION, 
	'limit': SEARCH_LIMIT,
    'offset': OFFSETN
	}


In [75]:
### Comment out all below after first run so list doesn't recreate itself.
#all_rest = []
#all_add = []
#all_city = []
#all_rating = []
#all_review = []

In [76]:


# Call the API
response = requests.request('GET', url, headers=headers, params=url_params)
 
for business in response.json()["businesses"]:
    all_rest.append(business["name"])
    all_review.append(business["review_count"]) 
    all_rating.append(business["rating"])
    all_add.append(business["location"]["address1"]) 
    all_city.append(business["location"]["city"]) 

 

In [77]:
#Combine data
final_yelp = all_rest + all_add + all_city + all_rating + all_review


In [78]:
#put into DataFrames
all_rest_df = pd.DataFrame(all_rest)
all_rating_df = pd.DataFrame(all_rating) 
all_add_df = pd.DataFrame(all_add) 
all_city_df = pd.DataFrame(all_city) 
all_review_df = pd.DataFrame(all_review) 

In [81]:
#merge all dataframes and rename columns
merge1 = all_rest_df.merge(all_add_df, how='outer', left_index=True, right_index=True)
merge1 = merge1.rename(columns={"0_x": "Name"})
merge1 = merge1.rename(columns={"0_y": "Address"})
merge2 = merge1.merge(all_city_df, how='outer', left_index=True, right_index=True)
merge2 = merge2.rename(columns={"0": "City"})
merge3 = merge2.merge(all_rating_df, how='outer', left_index=True, right_index=True)
merge4 = merge3.merge(all_review_df, how='outer', left_index=True, right_index=True)
merge4 = merge4.rename(columns={"Name": "Restaurant Name"})
merge4 = merge4.rename(columns={"Address": "Address"})
merge4 = merge4.rename(columns={"0_x": "City"})
merge4 = merge4.rename(columns={"0_y": "Rating"})
#the following is a fake merge.  I'm doing this to create an extra column, because before that, the last merge
#created the last column as "0" and it was impossible to rename.  So I force a merge to rename "0"
#to something I can rename, then I drop the last fake column.
final_merge = merge4.merge(all_add_df, how='outer', left_index=True, right_index=True)
final_merge = final_merge.rename(columns={"0_x": "Reviewers Count"})
final_merge.drop(["0_y"],axis = 1, inplace = True)


In [82]:
# append both dataframes, the CSV and the Yelp API data

final_etl_data = veg_data_df.append(final_merge, ignore_index =False, sort=True)
final_etl_data

,Address,City,Rating,Restaurant Name,Reviewers Count
0,"1511 High Street, Des Moines, IA 50309",Des Moines,4.4,A Dong Restaurant,659
1,"800 W 7th St, Sioux City, IA 51103",Sioux City,3.8,Da Kao Restaurant,182
2,"515 W 7th St, Sioux City, IA 51103",Sioux City,4.0,Diamond Thai Cuisine,303
3,"117 N Linn St, Iowa City, IA 52245",Cedar Rapids/Iowa City,4.0,Devotay,380
4,"211 10th Street, Augusta, GA 30901",Augusta,4.5,The Bees Knees,631
5,"415 Yellowstone Ave, Pocatello, ID 83201",Pocatello,3.6,Sushi Family,132
6,"5010 Armour Rd, Columbus, GA 31904",Columbus,4.2,Buckhead Bar and Grill,722
7,"199 Prince Ave, Athens, GA 30601",Athens,4.2,Grit,800
8,"11 N Orchard St, Boise, ID 83706",Boise,4.4,Lucianos Italian Restaurant,360
9,"2700 Dawson Rd, Albany, GA 31707",Albany,3.6,Hong Kong Cafe,88


In [85]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('ETL_Data.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
final_etl_data.to_excel(writer, sheet_name='Sheet1')

writer.save()